In [22]:
from Bending.FullBending import FullBending
import numpy as np
import pandas as pd

In [23]:
# E [GPa]
# nucleo	4.22
# X	26.95
# SX	13.57
# Y	97.86
# SY	4.22

df_layers = pd.DataFrame({ 'E': [4.01,
                                 6.02, 
                                 4.01, 
                                 14.13, 
                                 4.01]}, index=['nucleo', 'X', 'SX', 'Y', 'SY'])

In [24]:
df_layers

,E
nucleo,4.01
X,6.02
SX,4.01
Y,14.13
SY,4.01


In [25]:
composition = [
        ["Y", "Y"],
        ["Y", "X"],
        ["Y", "SX"],
        ["Y", "SY"],
        ["Y","SX","SY"],
        ["Y","X", "SY"]
    ]


In [26]:
from validation.BendingExperimental import BendingExperimental

r = BendingExperimental()

df_stats = r["df_stats"]


In [27]:
df_stats

Eb [GPa]              t [mm]               b [mm]            \
           mean       std      mean       std       mean       std   
Name                                                                 
TFP1  13.201050  0.630858  5.266667  0.057735  25.600000  0.100000   
TFP2   9.869501  0.780113  4.700000  0.000000  25.566667  0.057735   
TFP3   9.222003  0.495965  4.833333  0.057735  25.400000  0.173205   
TFP4  10.181815  0.770635  4.700000  0.000000  25.666667  0.152753   
TFP5   6.143105  0.161637  5.700000  0.000000  25.166667  0.152753   
TFP6   7.798060  0.334186  5.600000  0.000000  25.033333  0.635085   

           layers v [X,SX,Y,SY] ni  
                                    
Name                                
TFP1          [Y]  [0, 0, 1, 0]  1  
TFP2       [Y, X]  [1, 0, 1, 0]  2  
TFP3      [Y, SX]  [0, 1, 1, 0]  2  
TFP4      [Y, SY]  [0, 0, 1, 1]  2  
TFP5  [Y, SX, SY]  [0, 1, 1, 1]  3  
TFP6   [Y, X, SY]  [1, 0, 1, 1]  3

In [29]:

import os 
from copy import deepcopy
join = os.path.join

E_pred = []

if not os.path.exists("output"):
    os.mkdir("output")

params = {
            "geo" : {
                        "t_n" : 2.51,
                        "t_l" : 0.5,
                        "Lx"  : 30,
                    },
            "sim" : {
                        "disp" : 0.5,
                    },
        }


all_params = []

id_design = 0
E_n =   df_layers.loc["nucleo"].values[-1]*1e3

lyrs = df_stats["layers"].iloc[id_design]

E_l = df_layers.loc[lyrs]["E"].values*1e3 # MPa
name_mat = df_stats["layers"].iloc[id_design]
name = df_stats.index[id_design]
B    = df_stats["b [mm]"]["mean"].iloc[id_design]

params_loop = deepcopy(params)

B = 25
params_loop["geo"]["Ly"] = B
params_loop["sim"]["E_n"] = E_n
params_loop["sim"]["E_l"] = E_l
params_loop["sim"]["name_mat"] = name_mat
params_loop["output_folder"] = join("output")

print(params)
inp_f = FullBending(params_loop)
Eflex = params_loop["Eflex"]


{'geo': {'t_n': 2.51, 't_l': 0.5, 'Lx': 30}, 'sim': {'disp': 0.5}}
Running Calculix at:  output\sim
Output file:  c:\Users\djoroya\Documents\GitHub\CITISENS_Composite\scripts\T02_BendingTest\NumericalBasic\output\sim\out.txt
Command:
 mpiexec -n 4 c:\Users\djoroya\Documents\GitHub\CITISENS_Composite\.conda\Lib\site-packages\djccx\bin\ccx_dynamic.exe main
Error reading cvf file

pid:  3536 

Calculix finished



In [30]:
def E_flexion_an(E_n,E_l,t_n,t_l):

    t_t = t_n + 2*t_l

    yG = t_n*0.5 + t_l*0.5

    I_tot = (1/12)*t_t**3
    
    I_l   = (1/12)*t_l**3 + t_l*yG**2

    I_n   = (1/12)*t_n**3 


    Eflex = (E_n*I_n + 2*E_l*I_l)/(I_tot)

    return Eflex*1e-3 # GPa

In [31]:
from AnalyticalLayers.models import E_flexion  as E_flexion_fcn

In [34]:
t_n = params_loop["geo"]["t_n"]
t_l = params_loop["geo"]["t_l"]

Eflex_pred = E_flexion_an(E_n,
                          E_l[0],
                          t_n,
                          t_l)

Eflex_pred_fcn = E_flexion_fcn(E_n,
                           [E_l[0],0,0,0],
                           t_n,
                           t_l,layers=["X"])
Eflex_pred_fcn = Eflex_pred_fcn*1e-3

In [35]:
Eflex_pred - params_loop["Eflex"]

print("Analitico: ", Eflex_pred)
print("Simulado: ", Eflex)
print("Analitico fcn: ", Eflex_pred_fcn)    


Analitico:  10.429330271928873
Simulado:  10.851424148028935
Analitico fcn:  10.429330271928873
